In [1]:
# Goal: Data preprocessing, NLP, topic models with HDP (Gensim), visualizaion with pyLDAvis (preliminary)
# Result: implementations of topic models with high coherence scores from each library
# Output: pyLDAvis, wordcloud, csvs (document-topic, topic-document...)

# Load helper libraries
import re
from pprint import pprint
import pandas as pd
import numpy as np

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Enable Gensim logging and warnings
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# Text pre-processing model (lemmatization) from spacy (en) and nltk
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
#import seaborn
import wordcloud
#import sklearn
#import bokeh

In [2]:
# DATA Read in docs (corpus) from csv; report length of abstract text
docs = pd.read_csv("data/ERI-combined-2009-2019.csv")
len(docs)

# Concatenate title and abstract to new column for topic model
docs['combined'] = docs['title'].astype(str) + ' ' + docs['abstract'].astype(str)
docs.head()

# Calculate lengths of combined title and abstract; add columns to dataframe for length of abstracts
docs['title_len'] = docs['title'].apply(len)
docs['abstract_len'] = docs['abstract'].apply(len)
docs['combined_len'] = docs['combined'].apply(len)
docs.head()

# Calculate summary statistics for combined title and abstract lengths to determine text suitability
docs.combined_len.describe()

count    3770.000000
mean     1678.432626
std       687.609548
min       128.000000
25%      1247.000000
50%      1630.000000
75%      1993.000000
max      7083.000000
Name: combined_len, dtype: float64

In [3]:
# DATA Title and abstract to list, removing new line characters, quotations
#docs.head()
data = docs['combined'].values.tolist()
data = [re.sub('\s+', ' ', sent) for sent in data] #remove new line characters
data = [re.sub("\'", "", sent) for sent in data] #remove single quotes
pprint(data[:1])

<>:4: DeprecationWarning: invalid escape sequence \s
<>:4: DeprecationWarning: invalid escape sequence \s
<>:4: DeprecationWarning: invalid escape sequence \s
<ipython-input-3-7dc2a281053b>:4: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data] #remove new line characters


['Streams and Urbanization Urbanization encompasses a diverse array of '
 'watershed alterations that influence the physical, chemical, and biological '
 'characteristics of streams. In this chapter, we summarize lessons learned '
 'from the last half century of research on urban streams and provide a '
 'critique of various mitigation strategies, including recent approaches that '
 'explicitly address geomorphic processes. We focus first on the abiotic '
 'conditions (primarily hydrologic and geomorphic) and their changes in '
 'streams that accompany urbanization, recognizing that these changes may vary '
 'with geomorphic context and climatic region. We then discuss technical '
 'approaches and limitations to (1) mitigating water-quantity and '
 'water-quality degradation through site design, riparian protection, and '
 'structural stormwater-management strategies; and (2) restoring urban streams '
 'in those watersheds where the economic, social, and political contexts can '
 'supp

In [4]:
# NLP Load stopwords from NLTK, extend default list with custom stopwords
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['data', 'study', 'project', 'research', 'collaborative'])

/Users/saralafia/anaconda3/lib/python3.7/site-packages/nltk/decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""


In [5]:
# NLP Tokenize each sentence into a list of lowercase words
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True remove punctuation

data_words = list(sent_to_words(data))

print(data_words[:1])

[['streams', 'and', 'urbanization', 'urbanization', 'encompasses', 'diverse', 'array', 'of', 'watershed', 'alterations', 'that', 'influence', 'the', 'physical', 'chemical', 'and', 'biological', 'characteristics', 'of', 'streams', 'in', 'this', 'chapter', 'we', 'summarize', 'lessons', 'learned', 'from', 'the', 'last', 'half', 'century', 'of', 'research', 'on', 'urban', 'streams', 'and', 'provide', 'critique', 'of', 'various', 'mitigation', 'strategies', 'including', 'recent', 'approaches', 'that', 'explicitly', 'address', 'geomorphic', 'processes', 'we', 'focus', 'first', 'on', 'the', 'abiotic', 'conditions', 'primarily', 'hydrologic', 'and', 'geomorphic', 'and', 'their', 'changes', 'in', 'streams', 'that', 'accompany', 'urbanization', 'recognizing', 'that', 'these', 'changes', 'may', 'vary', 'with', 'geomorphic', 'context', 'and', 'climatic', 'region', 'we', 'then', 'discuss', 'technical', 'approaches', 'and', 'limitations', 'to', 'mitigating', 'water', 'quantity', 'and', 'water', 'qua

In [7]:
# NLP Bigram and trigram models (words frequently occurring together in the doc)
bigram = gensim.models.Phrases(data_words, min_count=3, threshold=80) # higher threshold fewer phrases
trigram = gensim.models.Phrases(bigram[data_words], threshold=80)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See bigram example (given a specific record)
print(trigram_mod[bigram_mod[data_words[1]]])

['evidence', 'for', 'widespread', 'creep', 'on', 'the', 'flanks', 'of', 'the', 'sea', 'of', 'marmara', 'transform', 'basin', 'from', 'marine', 'geophysical', 'data', 'wave', 'fields', 'have', 'long', 'been', 'recognized', 'in', 'marine', 'sediments', 'on', 'the', 'flanks', 'of', 'basins', 'and', 'oceans', 'in', 'both', 'tectonically_active', 'and', 'inactive', 'environments', 'the', 'origin', 'of', 'waves', 'hereafter', 'called', 'undulations', 'is', 'controversial', 'competing', 'models', 'ascribe', 'them', 'to', 'depositional', 'processes', 'gravity', 'driven', 'downslope', 'creep', 'or', 'collapse', 'and', 'or', 'tectonic', 'shortening', 'here', 'we', 'analyze', 'pervasive', 'undulation', 'fields', 'identified', 'in', 'swath', 'bathymetry', 'and', 'new', 'high', 'resolution', 'multichannel_seismic', 'mcs', 'reflection', 'data', 'from', 'the', 'sea', 'of', 'marmara', 'turkey', 'although', 'they', 'exhibit', 'some', 'of', 'the', 'classical', 'features', 'of', 'sediment', 'waves', 'the

In [8]:
# NLP Define functions for stopwords, bigrams, trigrams, and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [10]:
# NLP Remove stopwords, make bigrams, make trigrams, and lemmatize
data_words_nostops = remove_stopwords(data_words) # stop words
data_words_bigrams = make_bigrams(data_words_nostops) # bigrams
data_words_trigrams = make_trigrams(data_words_bigrams) # trigrams

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load('en', disable=['parser', 'ner'])

# Lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_trigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['stream', 'urbanization', 'urbanization', 'encompass', 'diverse', 'array', 'watershed', 'alteration', 'influence', 'physical', 'chemical', 'biological', 'characteristic', 'stream', 'chapter', 'summarize', 'lessons_learn', 'last', 'half', 'century', 'urban', 'stream', 'provide', 'critique', 'various', 'mitigation', 'strategy', 'include', 'recent', 'approach', 'explicitly', 'address', 'geomorphic', 'process', 'focus', 'first', 'abiotic', 'condition', 'primarily', 'hydrologic', 'geomorphic', 'change', 'stream', 'accompany', 'urbanization', 'recognize', 'change', 'may', 'vary', 'geomorphic', 'context', 'climatic', 'region', 'discuss', 'technical', 'approach', 'limitation', 'mitigate', 'water', 'quantity', 'water', 'quality', 'degradation', 'site', 'design', 'riparian', 'protection', 'structural', 'stormwater', 'management', 'strategy', 'restore', 'urban', 'stream', 'watershed', 'economic', 'social', 'political', 'context', 'support', 'activity']]


In [11]:
# NLP Create dictionary and corpus for topic modeling
# Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View corpus based on term-frequency
#print(corpus[:1])
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

# Save dictionary and corpus for future use - optional
#import pickle
#pickle.dump(corpus, open('corpus.pkl', 'wb'))
#id2word.save('dictionary.gensim')

[[('abiotic', 1),
  ('accompany', 1),
  ('activity', 1),
  ('address', 1),
  ('alteration', 1),
  ('approach', 2),
  ('array', 1),
  ('biological', 1),
  ('century', 1),
  ('change', 2),
  ('chapter', 1),
  ('characteristic', 1),
  ('chemical', 1),
  ('climatic', 1),
  ('condition', 1),
  ('context', 2),
  ('critique', 1),
  ('degradation', 1),
  ('design', 1),
  ('discuss', 1),
  ('diverse', 1),
  ('economic', 1),
  ('encompass', 1),
  ('explicitly', 1),
  ('first', 1),
  ('focus', 1),
  ('geomorphic', 3),
  ('half', 1),
  ('hydrologic', 1),
  ('include', 1),
  ('influence', 1),
  ('last', 1),
  ('lessons_learn', 1),
  ('limitation', 1),
  ('management', 1),
  ('may', 1),
  ('mitigate', 1),
  ('mitigation', 1),
  ('physical', 1),
  ('political', 1),
  ('primarily', 1),
  ('process', 1),
  ('protection', 1),
  ('provide', 1),
  ('quality', 1),
  ('quantity', 1),
  ('recent', 1),
  ('recognize', 1),
  ('region', 1),
  ('restore', 1),
  ('riparian', 1),
  ('site', 1),
  ('social', 1),
  

In [12]:
# TOPIC MODEL (HDP - Hierarchical Dirichlet Process)
# Fully unsupervised; infers the number of topics through "posterior inference"
# hdp_to_lda() Get corresponding alpha and beta values of a LDA almost equivalent to current HDP.
# suggested_lda_model() .

hdpmodel = gensim.models.HdpModel(corpus=corpus, id2word=id2word)
hdp_topics = hdpmodel.get_topics() #Get the term topic matrix learned during inference
len(hdp_topics) #150
hdpmodel.show_topics(num_topics=150) #Give the most probable num_words words from num_topics topics
hdp_lda = hdpmodel.suggested_lda_model() #Get a trained ldamodel object which is closest to the current hdp model
hdp_coherence = CoherenceModel(model=hdpmodel,texts=texts,dictionary=id2word,coherence='c_v').get_coherence()
print(hdp_coherence)

/Users/saralafia/anaconda3/lib/python3.7/site-packages/gensim/models/hdpmodel.py:467: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  start_time = time.clock()


0.4983231369481387


In [16]:
# TOPIC MODEL (LSI - Latent Semantic Indexing)
# LSI ranks topics by itself and outputs topics in ranked order; requires num_topics to be defined
lsimodel = gensim.models.LsiModel(corpus=corpus, num_topics=5, id2word=id2word) # Trying 10 topics as default
lsimodel.show_topics() 
lsitopics = lsimodel.print_topics()
#len(lsitopics)
pprint(lsitopics)
lsi_coherence = CoherenceModel(model=lsimodel,texts=texts,dictionary=id2word,coherence='c_v').get_coherence()
print(lsi_coherence)
#lsi_lda = lsimodel.suggested_lda_model() #Get a trained ldamodel object which is closest to the current lsi model

[(0,
  '0.343*"model" + 0.314*"use" + 0.198*"water" + 0.159*"high" + 0.157*"change" '
  '+ 0.141*"result" + 0.126*"increase" + 0.112*"specie" + 0.109*"base" + '
  '0.105*"scale"'),
 (1,
  '-0.739*"model" + 0.223*"water" + 0.196*"soil" + 0.161*"increase" + '
  '0.155*"plant" + 0.136*"change" + 0.135*"specie" + 0.126*"ecosystem" + '
  '0.103*"high" + 0.094*"effect"'),
 (2,
  '0.708*"water" + -0.333*"specie" + 0.169*"snow" + -0.137*"effect" + '
  '0.136*"use" + 0.122*"surface" + -0.122*"plant" + -0.115*"population" + '
  '-0.114*"model" + -0.106*"change"'),
 (3,
  '-0.379*"water" + -0.246*"model" + 0.203*"fault" + 0.197*"high" + '
  '-0.173*"specie" + 0.165*"sediment" + -0.159*"soil" + -0.144*"ecosystem" + '
  '0.144*"rate" + 0.141*"sample"'),
 (4,
  '0.485*"use" + 0.268*"specie" + -0.255*"model" + -0.250*"soil" + '
  '-0.215*"change" + -0.192*"climate" + -0.135*"rate" + 0.130*"method" + '
  '0.110*"base" + -0.108*"increase"')]
0.3733688463572714


In [14]:
# VISUALIZATION (HDP --> LDA), Gensim - Show model in pyLDAvis (150 topics)
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(hdp_lda, corpus, id2word)
#pyLDAvis.save_html(vis, 'pyLDAvis/hdp-gensim-150.html') #saves pyLDAvis graphs as standalone webpage
vis

/Users/saralafia/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.000015  0.000143       1        1  56.689706
1     -0.000172 -0.000049       2        1  23.202028
2      0.000035  0.000002       3        1   3.816326
3     -0.000070  0.000204       4        1   3.754612
4      0.000052 -0.000011       5        1   2.250592
5     -0.000016 -0.000082       6        1   1.878572
6     -0.000131 -0.000077       7        1   1.306463
9     -0.000177 -0.000130       8        1   0.712957
8     -0.000190 -0.000160       9        1   0.614132
7     -0.000259  0.000189      10        1   0.609110
10     0.000243 -0.000063      11        1   0.514884
12    -0.000143 -0.000169      12        1   0.281166
11    -0.000302  0.000358      13        1   0.243798
14     0.000288  0.000051      14        1   0.231196
15    -0.000190  0.000021      15        1   0.222608
13     0.000259  0.000111      16        1   0.187055
18    -0.000243  0.000354      17        1   0.143806
24     0.000039  0.000065      18        1   0.141539
35    -0.000018  0.000028      19        1   0.111789
27     0.000008 -0.000130      20        1   0.099896
37    -0.000070  0.000058      21        1   0.097143
31    -0.000365  0.000127      22        1   0.074984
23    -0.000067  0.000185      23        1   0.074607
17    -0.000022  0.000083      24        1   0.074060
19    -0.000045 -0.000165      25        1   0.073120
39    -0.000336  0.000344      26        1   0.072141
25     0.000060  0.000286      27        1   0.070678
22     0.000145  0.000127      28        1   0.068615
29     0.000336  0.000087      29        1   0.067364
28     0.000200 -0.000150      30        1   0.066476
...         ...       ...     ...      ...        ...
147   -0.000124 -0.000182     121        1   0.003956
103    0.000018  0.000009     122        1   0.003370
132    0.000320  0.000012     123        1   0.002782
115    0.000184 -0.000291     124        1   0.002758
135    0.000176  0.000003     125        1   0.002630
149    0.000100 -0.000104     126        1   0.002621
125    0.000180 -0.000084     127        1   0.002010
109   -0.000342  0.000039     128        1   0.000064
99    -0.000101  0.000045     129        1   0.000047
102    0.000258  0.000242     130        1   0.000045
122    0.000069  0.000123     131        1   0.000040
120   -0.000241 -0.000321     132        1   0.000032
116   -0.000040  0.000012     133        1   0.000032
118   -0.000034  0.000020     134        1   0.000031
119   -0.000035  0.000015     135        1   0.000029
121    0.000038  0.000118     136        1   0.000025
123   -0.000059  0.000295     137        1   0.000023
126   -0.000152  0.000066     138        1   0.000019
127   -0.000078 -0.000080     139        1   0.000018
129   -0.000210 -0.000082     140        1   0.000016
130   -0.000222 -0.000115     141        1   0.000015
131    0.000173  0.000041     142        1   0.000014
133    0.000079 -0.000174     143        1   0.000012
136    0.000139  0.000105     144        1   0.000012
137    0.000103 -0.000115     145        1   0.000009
141    0.000084  0.000024     146        1   0.000006
140    0.000109  0.000119     147        1   0.000006
142   -0.000335 -0.000153     148        1   0.000005
143   -0.000048 -0.000159     149        1   0.000004
145    0.000035  0.000204     150        1   0.000003

[150 rows x 5 columns], topic_info=       Category          Freq                      Term      Total  loglift  \
4430    Default  2.600000e+01                hypergraph  26.000000  30.0000   
1542    Default  2.500000e+01                   inhabit  25.000000  29.0000   
13499   Default  2.400000e+01                   diurnal  24.000000  28.0000   
36      Default  2.500000e+01                  mitigate  25.000000  27.0000   
12155   Default  2.500000e+01                    craton  25.000000  26.0000   
25682   Default  2.400000e+01                  ogallala  